In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import RFE, SelectKBest, SelectFromModel, chi2, f_classif, mutual_info_classif
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, ExtraTreesClassifier, AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from imblearn.over_sampling import SMOTE
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.impute import SimpleImputer
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from xgboost import XGBClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder

# List of specified features
specified_features = [
    'urg data hora admissao', 'IDADE', 'DIAS PRE OP',
    'urg_data_hora_admissao_mes', 'COD GDH', 'ORDEM PBASE DESC',
    'ANESTESIA', 'DIAS PRE OP_label', 'COD ESPEC DEST',
    'SEXO', 'TIPO ANESTESIA'
]

"""# Label Encoding for categorical columns
label_encoders = {}
for col in specified_features:
    if df[col].dtype == 'object' or df[col].dtype.name == 'category':
        le = LabelEncoder()
        df[col] = le.fit_transform(df[col])
        label_encoders[col] = le
"""
# Defining X and y
X = df[specified_features]
y = df["RISK_DEATH"]


# Creating an imputer that replaces NaN with the column's median
imputer = SimpleImputer(strategy='median')
X_imputed = imputer.fit_transform(X)
# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_imputed, y, test_size=0.25, random_state=42)

# Applying SMOTE to balance the classes
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)


# Create the scaler
scaler = MinMaxScaler()

# Apply normalization to the training and testing data
X_train_resampled_normalized = scaler.fit_transform(X_train_resampled)
X_test_normalized = scaler.transform(X_test)

# Defining the feature selection algorithms
feature_selectors = [
    ('RFE', RFE(estimator=RandomForestClassifier())),
    #('SelectKBest_chi2', SelectKBest(score_func=chi2)),
    #('SelectKBest_f_classif', SelectKBest(score_func=f_classif)),
    #('SelectKBest_mutual_info_classif', SelectKBest(score_func=mutual_info_classif)),
    ('SelectFromModel_RF', SelectFromModel(estimator=RandomForestClassifier())),
    ('SelectFromModel_GB', SelectFromModel(estimator=GradientBoostingClassifier())),
    ('SelectFromModel_LR', SelectFromModel(estimator=LogisticRegression())),
    ('SelectFromModel_ET', SelectFromModel(estimator=ExtraTreesClassifier())),
    ('SelectFromModel_AB', SelectFromModel(estimator=AdaBoostClassifier())),
]

# Defining the classification models
classifiers = [
    ('RandomForest', RandomForestClassifier()),
    ('GradientBoosting', GradientBoostingClassifier()),
    ('SVM', SVC()),
    ('DecisionTree', DecisionTreeClassifier()),
    #('LogisticRegression', LogisticRegression()),
    #('NaiveBayes', GaussianNB()),
    #('MLP', MLPClassifier()),
    ('XGBoost', XGBClassifier()),
    ('LGBM', LGBMClassifier()),
    ('CatBoost', CatBoostClassifier(silent=True)),
    ('ExtraTrees', ExtraTreesClassifier()),
    ('AdaBoost', AdaBoostClassifier()),
    ('RidgeClassifier', RidgeClassifier()),
]

# Creating pipelines to try different combinations of feature selection algorithms and classification models
for feature_selector_name, feature_selector in feature_selectors:
    for classifier_name, classifier in classifiers:
        pipeline = Pipeline([
            ('feature_selector', feature_selector),
            ('classifier', classifier)
        ])
        pipeline.fit(X_train_resampled_normalized, y_train_resampled)
        predictions = pipeline.predict(X_test_normalized)
        print(f"Feature Selector: {feature_selector_name}, Classifier: {classifier_name}")
        print(classification_report(y_test, predictions))
        print(confusion_matrix(y_test, predictions))